Install the requirements

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.7 MB/s eta 0:00:00


In [3]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.3 MB/s eta 0:00:00


In [4]:
pip install 'transformers[torch]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define perplexity functions to compute mean and std perplexity from a given list of sentences and input text file

In [8]:
import numpy as np
from evaluate import load
perplexity = load("perplexity", module_type="metric")

def compute_perplexity(sentences: str):
  results = perplexity.compute(model_id='gpt2', add_start_token=False, predictions=sentences)
  mean_p = results['mean_perplexity']
  std_p = np.std(results['perplexities'])
  return (mean_p, std_p)

In [9]:
def perplexity_from_file(fname):
  sentences = []
  with open(fname, 'r') as f:
    f.readline() # Skip the header
    for line in f:
      sentences.append(line.split('\t')[2])

  return compute_perplexity(sentences)

Test the perplexity function

In [10]:
with open('test.in', 'w') as f:
  f.write('input_sentiment\tprefix\tgenerated_tweet\n')
  f.write('negative\tThe main\tThe main character ran.\n')
  f.write('negative\tThe fox\tThe fox lept sadly.\n')

mean_p, std_p = perplexity_from_file('test.in')
print(mean_p, std_p)

Using pad_token, but it is not set yet.


  0%|          | 0/1 [00:00<?, ?it/s]

492.8201904296875 172.751953125


Run perplexity measurements on the actual data

In [12]:
baseline_mean_perplexity, baseline_std_perplexity = perplexity_from_file('baseline_generated_tweets.tsv')
custom_mean_perplexity, custom_std_perplexity = perplexity_from_file('custom_discriminator_generated_tweets.tsv')

print(f"Baseline mean perplexity: {baseline_mean_perplexity} +- {baseline_std_perplexity}")
print(f"Custom discriminator mean perplexity: {custom_mean_perplexity} +- {custom_std_perplexity}")

Using pad_token, but it is not set yet.


  0%|          | 0/63 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.


  0%|          | 0/63 [00:00<?, ?it/s]

Baseline mean perplexity: 18.992817537784575 +- 15.922139704353237
Custom discriminator mean perplexity: 19.760203359365462 +- 15.04595478505189
